$PE(pos, 2i) = sin(pos / 1000^{2i/d_{model}})$


$PE(pos, 2i+1) = cos(pos / 1000^{2i/d_{model}})$

> `i` = dimention index 

> `$d_{model}$` = Embedding length

> `pos` = Position of the word in the sequence

WHY? trianguler function...
- Periodicity
- Constrained values `[-1, +1]`
- Easy to extrapolate for long sequence

In [1]:
import torch
import torch.nn as nn

max_sequence_length = 20
d_model = 64 # Embedding dimension of model (d_model) is 512 on original paper

In [24]:
even_ids = torch.arange(0, max_sequence_length, 2).float()
odd_ids = torch.arange(1, max_sequence_length, 2).float()

In [25]:
even_ids

tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.])

In [27]:
odd_ids

tensor([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19.])

In [28]:
even_dnominator = torch.pow(10000, even_ids / d_model)
even_dnominator

tensor([ 1.0000,  1.3335,  1.7783,  2.3714,  3.1623,  4.2170,  5.6234,  7.4989,
        10.0000, 13.3352])

In [29]:
odd_denominator = torch.pow(10000, odd_ids / d_model)
odd_denominator

tensor([ 1.1548,  1.5399,  2.0535,  2.7384,  3.6517,  4.8697,  6.4938,  8.6596,
        11.5478, 15.3993])

In [30]:
denominator = even_dnominator

In [31]:
position = torch.arange(max_sequence_length).float().reshape(max_sequence_length, 1)
position

tensor([[ 0.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.],
        [11.],
        [12.],
        [13.],
        [14.],
        [15.],
        [16.],
        [17.],
        [18.],
        [19.]])

In [32]:
even_pe = torch.sin(position / denominator)
even_pe

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.8415,  0.6816,  0.5332,  0.4093,  0.3110,  0.2349,  0.1769,  0.1330,
          0.0998,  0.0749],
        [ 0.9093,  0.9975,  0.9021,  0.7469,  0.5911,  0.4567,  0.3482,  0.2636,
          0.1987,  0.1494],
        [ 0.1411,  0.7783,  0.9933,  0.9536,  0.8126,  0.6529,  0.5085,  0.3895,
          0.2955,  0.2231],
        [-0.7568,  0.1415,  0.7785,  0.9933,  0.9536,  0.8126,  0.6528,  0.5085,
          0.3894,  0.2955],
        [-0.9589, -0.5711,  0.3239,  0.8589,  0.9999,  0.9268,  0.7765,  0.6184,
          0.4794,  0.3662],
        [-0.2794, -0.9774, -0.2304,  0.5740,  0.9471,  0.9891,  0.8757,  0.7174,
          0.5646,  0.4349],
        [ 0.6570, -0.8593, -0.7137,  0.1886,  0.8004,  0.9960,  0.9473,  0.8037,
          0.6442,  0.5011],
        [ 0.9894, -0.2802, -0.9773, -0.2299,  0.5743,  0.9472,  0.9890,  0.8757,
          0.7174,  0.5646],
        [ 0.4121,  

In [35]:
odd_pe = torch.cos(position / odd_denominator)
print(odd_pe.shape)

torch.Size([20, 10])


In [36]:
stacked = torch.stack([even_pe, odd_pe], dim=2)
print(stacked.shape)
# stacked

torch.Size([20, 10, 2])


tensor([[[ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000]],

        [[ 0.8415,  0.6479],
         [ 0.6816,  0.7965],
         [ 0.5332,  0.8838],
         [ 0.4093,  0.9341],
         [ 0.3110,  0.9627],
         [ 0.2349,  0.9790],
         [ 0.1769,  0.9882],
         [ 0.1330,  0.9933],
         [ 0.0998,  0.9963],
         [ 0.0749,  0.9979]],

        [[ 0.9093, -0.1604],
         [ 0.9975,  0.2687],
         [ 0.9021,  0.5620],
         [ 0.7469,  0.7449],
         [ 0.5911,  0.8537],
         [ 0.4567,  0.9168],
         [ 0.3482,  0.9529],
         [ 0.2636,  0.9734],
         [ 0.1987,  0.9850],
         [ 0.1494,  0.9916]],

        [[ 0.1411, -0.8558],
         [ 0.7783, -0.3685],
         [ 0.9933,  0.1097],
         [ 0.9536,  0.4576],
        

In [38]:
PE = torch.flatten(stacked, start_dim=1, end_dim=2)
PE.shape

torch.Size([20, 20])

In [39]:
PE

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.6479,  0.6816,  0.7965,  0.5332,  0.8838,  0.4093,  0.9341,
          0.3110,  0.9627,  0.2349,  0.9790,  0.1769,  0.9882,  0.1330,  0.9933,
          0.0998,  0.9963,  0.0749,  0.9979],
        [ 0.9093, -0.1604,  0.9975,  0.2687,  0.9021,  0.5620,  0.7469,  0.7449,
          0.5911,  0.8537,  0.4567,  0.9168,  0.3482,  0.9529,  0.2636,  0.9734,
          0.1987,  0.9850,  0.1494,  0.9916],
        [ 0.1411, -0.8558,  0.7783, -0.3685,  0.9933,  0.1097,  0.9536,  0.4576,
          0.8126,  0.6811,  0.6529,  0.8162,  0.5085,  0.8952,  0.3895,  0.9406,
          0.2955,  0.9664,  0.2231,  0.9811],
        [-0.7568, -0.9485,  0.1415, -0.8556,  0.7785, -0.3682,  0.9933,  0.1099,
          0.9536,  0.4577,  0.8126,  0.6812,  0.6528,  0.8162,  0.5085,  0.8952,
      

# Positional Encoding Class

In [43]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_length, d_model):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_sequence_length = max_sequence_length
        self.register_buffer('PE', self._generate_PE())

    def _generate_PE(self):
        even_ids = torch.arange(0, self.d_model, 2).float()
        odd_ids = torch.arange(1, self.d_model, 2).float()

        even_denominator = torch.pow(10000, even_ids / self.d_model)
        odd_denominator = torch.pow(10000, odd_ids / self.d_model)

        position = torch.arange(self.max_sequence_length).float().reshape(self.max_sequence_length, 1)
        even_pe = torch.sin(position / even_denominator)
        odd_pe = torch.cos(position / odd_denominator)

        stacked = torch.stack([even_pe, odd_pe], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

    # def forward(self, x):
    #     assert x.size(1) <= self.max_sequence_length, f"Input sequence length is {x.size(1)} but PE is only defined up to {self.max_sequence_length}"
    #     assert x.size(2) == self.d_model, f"Input embedding dimension is {x.size(2)} but PE is only defined for d_model={self.d_model}"
    #     assert x.size(0) <= self.max_sequence_length, f"Batch size is {x.size(0)} but PE is only defined for batch size up to {self.max_sequence_length}"
    #     return x + self.PE[:x.size(0), :]

    def forward(self):
        return self.PE


In [48]:
positional_enc = PositionalEncoding(max_sequence_length=16, d_model=32)
p_e = positional_enc.forward()

In [49]:
p_e.shape

torch.Size([16, 32])